## This notebook compares performance of our models with original Baly's models

In [3]:
import classification as nmrc

In [ ]:
news_media_corpus_file = '''../News-Media_Reliability/data/corpus.csv'''
